In [1]:
import os
import networkx as nx
import random
import pickle
import math
#from mixmod_wt.Status import Status
#from mixmod_wt.new_modularity import __modularity
from collections import defaultdict
import matplotlib.pyplot as plt


In [7]:
def __renumber(dictionary) :
    count = 1
    #count = 0
    ret = dictionary.copy()
    new_values = dict([])

    for key in dictionary.keys() :
        value = dictionary[key]
        new_value = new_values.get(value, -1)
        if new_value == -1 :
            new_values[value] = count
            new_value = count
            count = count + 1
        ret[key] = new_value

    return ret


In [8]:
os.getcwd()

'/home/user/Downloads/sem2/mtp_prish/Louvain_mixmod'

In [10]:
dictionary = {1:1, 2:2, 3:4}
newdict = __renumber(dictionary)
print(newdict)

{1: 1, 2: 2, 3: 3}


In [18]:
#Multilayer network with 1 node in different layer
layer = {1:set([1,2,3])}
node_l = {1:set([2,3]), 2:set([3,1]), 3:set([1,2])}
node_c = {}
couple = {1:set([1,2,3])}
commu = {1:set([3,1,2])}
top = {1:1}
bot = {1:2}
graph = nx.Graph(); graph.add_edges_from([[1,2],[2,3],[3,1]])
print(graph.edges(), graph.nodes())
x1, x2, mod = simple_modularity(commu, graph)
print(mod, x1, x2)

[(1, 2), (1, 3), (2, 3)] [1, 2, 3]
modl:  -0.1111111111111111
modl:  -0.055555555555555546
modl:  1.3877787807814457e-17
modl:  0.05555555555555557
modl:  -0.05555555555555553
modl:  2.7755575615628914e-17
modl:  0.05555555555555559
modl:  0.11111111111111115
modl:  4.163336342344337e-17
6.938893903907228e-18 {1: 2.0816681711721685e-17} {1: 0.0}


In [33]:
layer = {1:set([1,2,3])}
node_l = {1:set([2,3]), 2:set([3,1]), 3:set([1,2])}
node_c = {}
couple = {1:set([1,2,3])}
commu = {1:set([1,2,3])}
top = {1:1}
bot = {1:2}
graph = nx.Graph(); graph.add_edges_from([[1,2],[2,3],[3,1]])
print(graph.edges(), graph.nodes())
x1, x2, mod = __smodularity(commu)
print(mod, x1, x2)

[(1, 2), (1, 3), (2, 3)] [1, 2, 3]
-0.9990000000000001 {1: -3.0} {1: 0}


In [24]:
modctr = 0

def __modularity(commu, graph):
    global modctr
    modctr += 1
    #print("modularity called", modctr, "edgewt: ", [graph[1][nbr].get('weight',1) for nbr in graph[1]])
    #print("From modularity, node_c: ", status.node_c)
    #print("From modularity, node_l: ", status.node_l)
    '''
    layer=status.layer
    node_l=status.node_l
    node_c=status.node_c       
    top=status.top
    bot=status.bot
    edge_l=status.edge_l
    edge_c=status.edge_c
    couple=status.couple
    '''
    mu = 1

    f=0    
    intra_inter={}
    for c in commu:
        intra_inter[c]=set()
        
        for n in commu[c]:
            for l in layer:
                if n in layer[l]:
                    intra_inter[c].add(l)
    
    #-----------------------------------------------------------------------------------    
    modularity=0    
    x1={}
    x2={}    
    
    for c in commu:
        x1[c]=0
        x2[c]=0
        modc_layer=0
        if len(intra_inter[c])>1:
            for l in layer:
                m_layer = 0.0
                d_layer=0.0
                I_layer=0.0
                n_layer=0.0
                n_co_com_layer=0.0

                for n in layer[l]:
                    if n in node_l:
                        m_layer += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                        #m_layer+=len(node_l[n])

                    if n in commu[c]:
                        n_layer+=1

                        if n in node_l:
                            d_layer += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                            #d_layer+=len(node_l[n])

                            for nei in node_l[n]:
                                if nei in commu[c]: #if the neighbour belongs to current community
                                    I_layer += graph[n][nei].get('weight',1)
                                    #I_layer+=1
                        #----useless code block------
                        if n in node_c:
                            for nei in node_c[n]:
                                if nei not in commu[c]: #connected to atleast one crosslayer node outside community
                                    n_co_com_layer+=1 
                                    break
                        #----------------------------
                                            
                I_layer=float(I_layer)/2.0
                m_layer=float(m_layer)/2.0
                if I_layer>-1 and m_layer>0:
                    mod=((I_layer/m_layer)-((float(d_layer)/(2*m_layer))*(float(d_layer)/(2*m_layer))))
                else:
                    mod=0    
                if f==1:# f is always 0
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
            #----------------------------------------------------------------------
            modc_couple=0
            for co in couple:
                d_couple_top=0.0
                d_couple_bot=0.0
                I_couple=0.0
                m_couple=0.0
            
                top_tot=0.0
                top_con=0.0
                bot_tot=0.0
                bot_con=0.0
                
                m_layer_top=0.0
                m_layer_bot=0.0
                
                for n in couple[co]:
                    if n in node_c:
                        for nei in node_c[n]:
                            if (n in layer[top[co]] and nei in layer[bot[co]]) or (n in layer[bot[co]] and nei in layer[top[co]]):
                                m_couple += graph[n][nei].get('weight',1)
                                #m_couple+=1
                
                    if n in layer[top[co]]: #n belongs to the top layer of coupling
                        if n in node_l:
                            m_layer_top += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                            #m_layer_top+=len(node_l[n])
                            
                        if n in commu[c]:    #if the node belongs to current community
                            top_tot+=1
                            if n in node_c:
                                flagg=0
                                for nei in node_c[n]:
                                    if nei in layer[bot[co]]:
                                        d_couple_top += graph[n][nei].get('weight',1)
                                        #d_couple_top += 1
                                        if nei in commu[c]: #if the neighbour belongs to current community
                                            I_couple += graph[n][nei].get('weight',1)
                                            #I_couple+=1
                                            flagg=1
                                if flagg==1: #connected to at least 1 within community node in bottom layer
                                    top_con+=1
                            if n in node_l and n not in node_c:
                                d_couple_top += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                                #d_couple_top+=len(node_l[n])
                                    
                                            
                    if n in layer[bot[co]]: #n belongs to the bottom layer of coupling
                        if n in node_l:
                            m_layer_bot += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                            #m_layer_bot+=len(node_l[n])
                        if n in commu[c]:    #if the node belongs to current community
                            bot_tot+=1
                            if n in node_c:
                                flagg=0
                                for nei in node_c[n]:
                                    if nei in layer[top[co]]:
                                        d_couple_bot += graph[n][nei].get('weight',1)
                                        #d_couple_bot+=1    
                                        if nei in commu[c]: #if the neighbour belongs to current community
                                            flagg=1                    
                                                #break
                                if flagg==1: #connected to at least 1 within community node in bottom layer
                                    bot_con+=1
                            if n in node_l and n not in node_c:
                                d_couple_bot += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                                #d_couple_bot+=len(node_l[n])        
                                
                I_couple=float(I_couple)
                m_couple=float(m_couple)/2.0
                m_layer_top=float(m_layer_top)/2.0
                m_layer_bot=float(m_layer_bot)/2.0
                
                if I_couple>-1 and (m_couple+m_layer_top+m_layer_bot)>0:
                    
                    mod=((I_couple/(m_couple+m_layer_top+m_layer_bot))-((d_couple_top*d_couple_bot)/((m_couple+2.0*m_layer_top+2.0*m_layer_bot)*(m_couple+2.0*m_layer_top+2.0*m_layer_bot))))
                else:
                    mod=0    
                ##print I_couple, m_couple, d_couple_top, d_couple_bot    
                #print c,co,mod,mu,2*(1-mu)*mod,I_couple,m_couple,d_couple_top,d_couple_bot
                if f==1:
                    modc_couple+=2*(1-mu)*mod
                else:
                    modc_couple+=mod
                    x2[c]+=mod
                #print modc_couple            
                ##print "ha hh"
            #-----------------------------------------------------------------------
            modularity+=modc_layer+modc_couple
            
        else:
            l=list(intra_inter[c])
            #print("l: ",l, " c: ", c)
            l=l[0];
            d_layer=0.0; d_layer1 = 0.0;
            I_layer=0.0; I_layer1 = 0.0;
            m_layer=0.0; m_layer1 = 0.0
            n_layer=0.0
            n_co_com_layer=0.0
            c_layer=0.0; c_layer1 = 0.0


            for n in layer[l]:
                
                if n in node_l:
                    m_layer += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                    #m_layer+=len(node_l[n])
                if n in node_c:
                    c_layer += sum([graph[n][nbr].get('weight',1) for nbr in node_c[n]])
                    #c_layer+=len(node_c[n])

            for n in commu[c]:
                #n_layer+=1
                if n in node_l:
                    d_layer += sum([graph[n][nbr].get('weight',1) for nbr in node_l[n]])
                    #d_layer+=len(node_l[n])

                    for nei in node_l[n]:
                        if nei in commu[c]: #if the neighbour belongs to current community
                            I_layer += graph[n][nei].get('weight',1)
                            #I_layer+=1
                if n in node_c and len(node_c[n])>0:
                    n_co_com_layer+=1 

            #print("I: ",I_layer, I_layer1)
            #---------------------------------------------------------
            #--------no modifications done after this point-----------   
            #---------------------------------------------------------  
            I_layer=float(I_layer)/2.0
            m_layer=float(m_layer)/2.0
            if I_layer>-1 and m_layer>0:
                mod=((I_layer/(1.0*m_layer))-((float(d_layer)/(1.0*m_layer))*(float(d_layer)/(1.0*m_layer))))   
            else:
                mod=0 
            ##print c,l,mod,mu,mu*mod,I_layer,m_layer,c_layer,d_layer, n_co_com_layer, n_layer
            if f==1: #f is always 0
                modc_layer+=mu*mod
            else:    
                modc_layer+=mod
                x1[c]+=mod

            modularity+=modc_layer        
                                    
    ##print x1,x2    
    return x1, x2, 0.333*modularity    



In [32]:
def __smodularity(commu):
    '''
    layer=status.layer
    node_l=status.node_l
    node_c=status.node_c       
    top=status.top
    bot=status.bot
    edge_l=status.edge_l
    edge_c=status.edge_c
    couple=status.couple
    '''
    mu = 1

    f=0    
    intra_inter={}
    for c in commu:
        intra_inter[c]=set()
        
        for n in commu[c]:
            for l in layer:
                if n in layer[l]:
                    intra_inter[c].add(l)
    
    #print intra_inter
    
    modularity=0    
    x1={}
    x2={}    
    for c in commu:
        x1[c]=0
        x2[c]=0
        modc_layer=0
        
        if len(intra_inter[c])>1:
            for l in layer:
                d_layer=0.0
                I_layer=0.0
                m_layer=0.0
                n_layer=0.0
                n_co_com_layer=0.0
            
                for n in layer[l]:
                    if n in node_l:
                        m_layer+=len(node_l[n])
                    #if n in node_c:
                    #    m_layer+=len(node_c[n])    
                    if n in commu[c]:    #if the node belongs to current community
                        n_layer+=1
                        if n in node_l:
                            d_layer+=len(node_l[n])
                    
                            for nei in node_l[n]:
                                if nei in commu[c]: #if the neighbour belongs to current community
                                    I_layer+=1
                    
                        if n in node_c:
                            for nei in node_c[n]:
                                if nei not in commu[c]: #connected to atleast one crosslayer node outside community
                                    n_co_com_layer+=1 
                                    break
                        '''
                            for nei in node_c[n]:
                                if nei in commu[c]: #if the neighbour belongs to current community
                                    I_layer+=1
                        '''
                                            
                I_layer=float(I_layer)/2.0
                m_layer=float(m_layer)/2.0x1, x2,
                if I_layer>-1 and m_layer>0:
                    mod=((I_layer/m_layer)-((float(d_layer)/(2*m_layer))*(float(d_layer)/(2*m_layer))))
                else:
                    mod=0    
                #if n_layer > 0:
                #    mod=mod*pow(2.718,-(float(n_co_com_layer)/float(n_layer)))
            
                #print mod    
                if f==1:
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
            
            modc_couple=0
            for co in couple:
                d_couple_top=0.0
                d_couple_bot=0.0
                I_couple=0.0
                m_couple=0.0
            
                top_tot=0.0
                top_con=0.0
                bot_tot=0.0
                bot_con=0.0
                
                m_layer_top=0.0
                m_layer_bot=0.0
                
                for n in couple[co]:
                    if n in node_c:
                        for nei in node_c[n]:
                            if (n in layer[top[co]] and nei in layer[bot[co]]) or (n in layer[bot[co]] and nei in layer[top[co]]):
                                m_couple+=1
                
                    if n in layer[top[co]]: #n belongs to the top layer of coupling
                        if n in node_l:
                            m_layer_top+=len(node_l[n])
                            
                        if n in commu[c]:    #if the node belongs to current community
                            top_tot+=1
                            if n in node_c:
                                flagg=0
                                for nei in node_c[n]:
                                    if nei in layer[bot[co]]:
                                        d_couple_top+=1
                                        if nei in commu[c]: #if the neighbour belongs to current community
                                            I_couple+=1
                                            flagg=1
                                if flagg==1: #connected to at least 1 within community node in bottom layer
                                    top_con+=1
                            if n in node_l and n not in node_c:
                                d_couple_top+=len(node_l[n])
                                    
                                            
                    if n in layer[bot[co]]: #n belongs to the bottom layer of coupling
                        if n in node_l:
                            m_layer_bot+=len(node_l[n])
                        if n in commu[c]:    #if the node belongs to current community
                            bot_tot+=1
                            if n in node_c:
                                flagg=0
                                for nei in node_c[n]:
                                    if nei in layer[top[co]]:
                                        d_couple_bot+=1    
                                        if nei in commu[c]: #if the neighbour belongs to current community
                                            flagg=1                    
                                                #break
                                if flagg==1: #connected to at least 1 within community node in bottom layer
                                    bot_con+=1
                            if n in node_l and n not in node_c:
                                d_couple_bot+=len(node_l[n])        
                                
                I_couple=float(I_couple)
                m_couple=float(m_couple)/2.0
                m_layer_top=float(m_layer_top)/2.0
                m_layer_bot=float(m_layer_bot)/2.0
                
                if I_couple>-1 and (m_couple+m_layer_top+m_layer_bot)>0:
                    
                    mod=((I_couple/(m_couple+m_layer_top+m_layer_bot))-((d_couple_top*d_couple_bot)/((m_couple+2.0*m_layer_top+2.0*m_layer_bot)*(m_couple+2.0*m_layer_top+2.0*m_layer_bot))))
                else:
                    mod=0    
                #print I_couple, m_couple, d_couple_top, d_couple_bot    
                if f==1:
                    modc_couple+=2*(1-mu)*mod
                else:
                    modc_couple+=mod
                    x2[c]+=mod
                #print "ha hh"
            modularity+=modc_layer+modc_couple
            
        else:
            for l in layer:
                d_layer=0.0
                I_layer=0.0
                m_layer=0.0
                n_layer=0.0
                n_co_com_layer=0.0
            
                for n in layer[l]:
                    if n in node_l:
                        m_layer+=len(node_l[n])
                
                c_layer=0.0    
                for n in layer[l]:
                    if n in node_c:
                        c_layer+=len(node_c[n])
                            
                    #if n in node_c:
                    #    m_layer+=len(node_c[n])    
                    if n in commu[c]:    #if the node belongs to current community
                        n_layer+=1
                        if n in node_l:
                            d_layer+=len(node_l[n])
                    
                            for nei in node_l[n]:
                                if nei in commu[c]: #if the neighbour belongs to current community
                                    I_layer+=1
                    
                        if n in node_c and len(node_c[n])>0:
                            n_co_com_layer+=1 
                            #d_layer+=len(node_c[n])
                            
                        
                                            
                I_layer=float(I_layer)/2.0
                m_layer=float(m_layer)/2.0
                
                if I_layer>-1 and m_layer>0:#(1.0*m_layer+(c_layer/2.0))>0:# and m_layer1>0:
                    
                    #mod=((I_layer/(1.0*m_layer+(c_layer/2.0)))-((float(d_layer)/(2.0*m_layer+c_layer))*(float(d_layer)/(2.0*m_layer+c_layer))))
                    mod=((I_layer/(1.0*m_layer))-((float(d_layer)/(1.0*m_layer))*(float(d_layer)/(1.0*m_layer))))
                    #else:
                    #mod=(-((d_layer/(2*m_layer))*(d_layer/(2*m_layer))))    
                else:
                    mod=0 
                #if n_layer > 0:
                #    mod=mod*pow(2.718,-((float(n_co_com_layer))/float(n_layer)))
            
                #print mod    
                if f==1:
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
            modularity+=modc_layer        
                                    
    return x1, x2, 0.333*modularity    




In [26]:
def simple_modularity(commu, graph):
	#layer = status.layer 
	#node_l = status.node_l
	#node_c = status.node_c

	nodelayer = {}
	for l in layer:
		for node in layer[l]:
			nodelayer[node] = l

	x1 = {}	#stores within layer modularity values
	x2 = {}	#store coupling modularity values

	#graph[n][nei].get('weight',1)
	layerdegreesum = {}
	for l in layer:
		layerdegreesum[l] = 0
		for n1 in layer[l]:
			for n2 in node_l.get(n1, set()):
				layerdegreesum[l] += graph[n1][n2].get('weight', 1)

	couplingdegreesum = 0; #print("layerdegreesum: ",layerdegreesum)

	for n1 in node_c:
		for n2 in node_c[n1]:
			   	couplingdegreesum += graph[n1][n2].get('weight',1)

	#Compute modularity value for each community separately
	modularity = 0
	for c in commu:
		x1[c] = 0
		x2[c] = 0
		if(len(commu[c])==1): modl = 0 
		else:
			for l in layer:
				modl = 0
				for n1 in commu[c]:
					d1 = 0
					for n3 in node_l[n1]:
						d1 += graph[n1][n3].get('weight',1)

					for n2 in commu[c]:
						aij = 0
                        d2 = 0
                        if(nodelayer[n2]!=nodelayer[n1]): continue
                        
                        if((n1==n2 and n2 in node_l(n1)) or n1!=n2):
                            if(n2 in node_l[n1]):
                                if(n1!=n2): aij = graph[n1][n2].get('weight',1)
                                else: aij = graph[n1][n2].get('weight',1)

                            for n3 in node_l[n2]:
                                d2 += graph[n2][n3].get('weight',1)
						
						modl += (aij*1.0-(d1*d2*1.0)/layerdegreesum[l])/layerdegreesum[l]; print("modl: ",modl)

			x1[c] += modl*1.0/2

		modc = 0
		for n1 in commu[c]:
			d1 = 0
			for n3 in node_c.get(n1, set()):
				d1 += graph[n1][n3].get('weight',1)
			
			for n2 in commu[c]:
				if(n1==n2): continue
				if(n2 not in node_c.get(n1, set())): continue

				d2 = 0
				for n3 in node_c.get(n2, set()):
					d2 += graph[n2][n3].get('weight',1)

				aij = graph[n1][n2].get('weight',1)
					
				if(couplingdegreesum == 0): 
					modc = 0
				else:
					modc += (aij*1.0-((d1*d2*1.0)/couplingdegreesum))/couplingdegreesum

		x2[c] = modc/2;
		modularity += x1[c] + x2[c]

	return x1, x2, (1.0/3)*modularity


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-26-3b75f1076f8a>, line 44)

In [9]:
#Multilayer network with 1 node in different layer
layer = {1:set([1,2,3])}
node_l = {1:set([2,3]), 2:set([3,1]), 3:set([1,2])}
node_c = {}
couple = {1:set([1,2,3])}
commu = {1:set([1,2,3])}
top = {1:1}
bot = {1:2}
graph = nx.Graph(); graph.add_edges_from([[1,2],[2,3],[3,1]])
print(graph.edges(), graph.nodes())
x1, x2, mod = correct_new_weighted_correct_implementation(commu, graph)
print(mod, x1, x2)

([(1, 2), (1, 3), (2, 3)], [1, 2, 3])
(0.0555, {1: 0.16666666666666666}, {1: 0})


In [8]:
def correct_new_weighted_correct_implementation(commu,graph):
    
    mu = 1

    #Construct a dict = {node1: layer, node2: layer......}
    nodelayer = {}
    for l in layer:
        for node in layer[l]:
            nodelayer[node] = l

    #calculate Intra_inter ----------------------------------------------------------
    f=0    
    intra_inter={}
    for c in commu:
        intra_inter[c]=set()
        
        for n in commu[c]:
            for l in layer:
                if n in layer[l]:
                    intra_inter[c].add(l)

    #--------------------------------------------------------------------------------

    #calculate |E1| , |E2| , |E12|---------------------------------------------------
    E={}
    E12=0
    for l in layer:
        E[l]=0
        for n in layer[l]:
            for nei in node_l.get(n,set()):
                E[l]+= graph[n][nei].get('weight',1)
        E[l] = E[l]/2

    for n in node_c:
        for nei in node_c[n]:
            E12+=graph[n][nei].get('weight',1)
    E12 = E12/2
    #--------------------------------------------------------------------------------

   
    modularity=0    
    x1={}
    x2={}    
    
    for c in commu:
        x1[c]=0
        x2[c]=0
        modc_layer=0
        if len(intra_inter[c])>1:
            for l in layer:
                Aij=0.0
                hihj=0.0
                
                #compute summation Aij------------------------
                for n in commu[c]:
                    if n in layer[l]:
                        for nei in node_l.get(n,set()):
                            if nei in commu[c]:
                                Aij+=graph[n][nei].get('weight',1)
                Aij = Aij/2

                #compute summation hihj--------------------------
                for n1 in commu[c]:
                    if n1 in layer[l]:
                        for n2 in commu[c]:
                            if n2 in layer[l]:
                                if(n1==n2):
                                    if(n1 not in node_l[n1]): 
                                        continue
                                hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                                hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                                hihj+= (hi*hj)
                hihj = hihj/2
                #-------------------------------------------------
                try:
                    mod = (1.0/(2*E[l]))*(Aij - (hihj*1.0/(2*E[l])))
                except:
                    mod=0
                
                if f==1:# f is always 0
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
        #----------------------------------------------------------------------
            modc_couple=0
            Aij=0
            cicj=0
            
            #compute Aij------------------------------
            for n in commu[c]:
                if n in node_c:
                    for nei in node_c[n]:
                        if nei in commu[c]:
                            Aij+= graph[n][nei].get('weight',1)
            Aij = Aij/2

            #compute cicj-----------------------------
            for n1 in commu[c]:
                for n2 in commu[c]:
                    if(n1==n2 or nodelayer[n1]==nodelayer[n2] ) :
                        #If nodes in same layer, continue
                        continue
                    if n1 in node_c:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c[n1]])
                    else:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                    if n2 in node_c:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c[n2]])
                    else:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])

                cicj += (ci*cj)
            cicj = cicj/2

            try:
                norm = 1.0/(2*sum([E[l] for l in layer]) + 2*E12) #norm = 1/(2*|E1| + 2*|E2| + 2*|E12|)
                mod = norm*(Aij - (norm*cicj))
            except:
                mod=0
            
            if f==1:
                modc_couple+=2*(1-mu)*mod
            else:
                modc_couple+=mod
                x2[c]+=mod
            #print modc_couple            
            ##print "ha hh"
            #-----------------------------------------------------------------------
            modularity+=modc_layer+modc_couple
            
        else:
            l=list(intra_inter[c])
            l=l[0];
            Aij=0.0
            hihj=0.0
            
            #compute summation AIJ------------------------
            for n in commu[c]:
                for nei in node_l.get(n,set()):
                    if nei in commu[c]:
                        Aij+=graph[n][nei].get('weight',1)
            Aij = Aij/2

            #compute summation hihj--------------------------
            for n1 in commu[c]:
                for n2 in commu[c]:
                    if(n1==n2):
                        if(n1 not in node_l[n1]):
                            continue
                    hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                    ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c.get(n1,set())])
                    hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                    cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c.get(n2,set())])
                    hihj+= ((hi+ci)*(hj+cj))

            hihj = hihj/2
            #-------------------------------------------------
            try:
                norm = 1.0/(2*E[l] + 2*E12)
                mod = norm*(Aij*1.0 - (norm*hihj))
            except:
                mod=0
            
            if f==1: #f is always 0
                modc_layer+=mu*mod
            else:    
                modc_layer+=mod
                x1[c]+=mod

            modularity+=modc_layer        
                                    
    ##print x1,x2    
    return x1, x2, 0.333*modularity    
